In [1]:
%run ./config_All_Zones_A

In [2]:
companies = spark.read.format('delta').load("/mnt/silver/cab/companies").selectExpr("CAST(CompanyId AS String) AS CompanyId","CAST(AccountId AS String) AS AccountId","CAST(TierId AS String) AS TierId","CAST(InstanceId AS String) AS InstanceId","IsActive","Date(SetupCompletionTime) AS SetupCompletionTime")
accounts = spark.read.format('delta').load("/mnt/silver/cab/accounts").selectExpr("CAST(BillingAccountId AS String) AS AccountId","Status","IsTest","CreationDate","industrycode AS IndustryCode")
licenses = spark.read.format('delta').load("/mnt/silver/cab/licenses").selectExpr("CAST(LicenseId AS String) AS LicenseId", "CAST(AccountId AS String) AS AccountId", "CAST(CompanyId AS String) AS CompanyId", "InstanceId", "TierId", "Count AS LicenseCount","CreatedOn","OriginalActivationDate AS Activated","DeactDate AS Deactivated")

In [3]:
import pandas as pd
#get non-test registered accounts/companies
billable_accounts = accounts.filter(accounts.IsTest == False)
registered_companies = companies.filter(companies.SetupCompletionTime.isNotNull())
billable_registered_companies = billable_accounts.join(registered_companies, on = ['AccountId'], how = 'left').toPandas()
billable_registered_companies['CompanyId'] = billable_registered_companies['CompanyId'].astype("string")
print(billable_registered_companies)

AccountId Status IsTest ... InstanceId IsActive SetupCompletionTime
0 375 1 False ... 50 True 2015-09-29
1 384 1 False ... 50 True 2015-09-30
2 497 1 False ... 50 True 2015-10-28
3 593 1 False ... 50 True 2015-12-09
4 623 1 False ... 50 True 2015-12-16
... ... ... ... ... ... ... ...
11667 20039 1 False ... 50 True 2020-06-26
11668 20076 1 False ... None None None
11669 20091 1 False ... 50 True 2020-07-06
11670 20103 1 False ... None None None
11671 20126 1 False ... None None None

[11672 rows x 10 columns]

In [5]:
trips = spark.read.format('delta').load("/mnt/silver/cab/trips").selectExpr("CAST(InstanceId AS String) AS InstanceId","CAST(CompanyId AS String) AS CompanyId","DeviceId","Date(StartUtcTime) AS UsageDate", "1 AS Trip_Count").groupBy(['InstanceId','CompanyId','UsageDate']).agg({'Trip_Count':'sum'}).withColumnRenamed("sum(Trip_Count)", "Trip_Count")
form_headers = spark.read.format('delta').load("/mnt/silver/cab/form_headers").selectExpr("CAST(InstanceId AS String) AS InstanceId","CAST(CompanyId AS String) AS CompanyId","Date(CreatedUtc) AS UsageDate", "StartedByUserId AS Form_UserId", "1 AS Form_Count").groupBy(['InstanceId','CompanyId','UsageDate']).agg({'Form_Count':'sum'}).withColumnRenamed("sum(Form_Count)", "Form_Count")
timekeeping_statuses = spark.read.format('delta').load("/mnt/silver/cab/timekeeping_statuses").selectExpr("CAST(InstanceId AS String) AS InstanceId","CAST(CompanyId AS String) AS CompanyId","Date(StartUtcTime) AS UsageDate", "UserId AS Status_UserId", "1 AS Status_Count").groupBy(['InstanceId','CompanyId','UsageDate']).agg({'Status_Count':'sum'}).withColumnRenamed("sum(Status_Count)", "Status_Count")
order_headers = spark.read.format('delta').load("/mnt/silver/cab/order_headers").selectExpr("CAST(InstanceId AS String) AS InstanceId","CAST(CompanyId AS String) AS CompanyId","Date(CreatedUTCTime) AS UsageDate", "CreatedByUserId AS Order_UserId", "1 AS Order_Count").groupBy(['InstanceId','CompanyId','UsageDate']).agg({'Order_Count':'sum'}).withColumnRenamed("sum(Order_Count)", "Order_Count")
#incidents = spark.read.format('delta').load("/mnt/silver/dynamics/incidents")
messages = spark.read.format('delta').load("/mnt/silver/cab/sent_messages/").selectExpr("CAST(InstanceId AS String) AS InstanceId","CAST(CompanyId AS String) AS CompanyId","Date(SentTimeUtc) AS UsageDate", "FromUserId AS Message_UserId", "1 AS Message_Count").groupBy(['InstanceId','CompanyId','UsageDate']).agg({'Message_Count':'sum'}).withColumnRenamed("sum(Message_Count)", "Message_Count")

In [6]:
merged_usage = trips.join(form_headers, on = ['InstanceId', 'CompanyId', 'UsageDate'], how = 'outer')
merged_usage = merged_usage.join(order_headers, on = ['InstanceId', 'CompanyId', 'UsageDate'], how = 'outer')
merged_usage = merged_usage.join(timekeeping_statuses, on = ['InstanceId', 'CompanyId', 'UsageDate'], how = 'outer')
merged_usage = merged_usage.join(messages, on = ['InstanceId', 'CompanyId', 'UsageDate'], how = 'outer')
merged_usage = merged_usage.fillna(0, ['Trip_Count', 'Form_Count', 'Order_Count', 'Status_Count', 'Message_Count']).toPandas()
display(merged_usage)

InstanceId,CompanyId,UsageDate,Trip_Count,Form_Count,Order_Count,Status_Count,Message_Count
50,1001,2016-06-29,8,0,0,0,0
50,1001,2016-08-19,3,0,0,0,0
50,10011,2017-10-30,2,0,0,0,0
50,10011,2017-12-05,4,0,0,0,0
50,10011,2017-12-07,5,0,0,0,0
50,10011,2018-01-07,2,0,0,0,0
50,10011,2018-01-26,3,0,0,0,0
50,10011,2018-01-31,4,0,0,0,0
50,10011,2018-03-27,3,0,0,0,0
50,10011,2018-05-17,4,0,0,0,0


In [7]:
#result = df.sort(['A', 'B'], ascending=[1, 0])
print(merged_usage['UsageDate'].sort_values(ascending = False))

298552 2560-04-19
109496 2020-09-10
188719 2020-09-10
739281 2020-09-10
426470 2020-09-10
 ... 
107118 2010-01-01
719952 2010-01-01
68076 2010-01-01
1067588 2009-01-01
555428 2004-01-20
Name: UsageDate, Length: 1281364, dtype: object

In [8]:
print(billable_registered_companies.dtypes)
filtered_merged_usage = pd.merge(billable_registered_companies, merged_usage, on = ['CompanyId'], how = 'inner')


--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1306202716739055> in <module> 
 1 print ( billable_registered_companies . dtypes ) 
 ----> 2 filtered_merged_usage = pd . merge ( billable_registered_companies , merged_usage , on = [ 'CompanyId' ] , how = 'inner' ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in merge (left, right, how, on, left_on, right_on, left_index, right_index, sort, suffixes, copy, indicator, validate) 
 86 validate = validate , 
 87 )
 ---> 88 return op . get_result ( ) 
 89 
 90 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in get_result (self) 
 641 self . left , self . right = self . _indicator_pre_merge ( self . left , self . right ) 
 642 
 --> 643 join_index , left_indexer , right_indexer = self . _get_join_info ( ) 
 644 
 645 ldata , rdata = self . left . _data , self . right . _data

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in _get_join_info (self) 
 860 )
 861 else : 
 --> 862 ( left_indexer , right_indexer ) = self . _get_join_indexers ( ) 
 863 
 864 if self . right_index : 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in _get_join_indexers (self) 
 839 """ return the join indexers """ 
 840 return _get_join_indexers(
 --> 841 self . left_join_keys , self . right_join_keys , sort = self . sort , how = self . how
 842 )
 843 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in _get_join_indexers (left_keys, right_keys, sort, how, **kwargs) 
 1309 for n in range ( len ( left_keys ) ) 
 1310 )
 -> 1311 zipped = zip ( * mapped ) 
 1312 llab , rlab , shape = [ list ( x ) for x in zipped ] 
 1313 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in <genexpr> (.0) 
 1307 mapped = (
 1308 _factorize_keys ( left_keys [ n ] , right_keys [ n ] , sort = sort ) 
 -> 1309 for n in range ( len ( left_keys ) ) 
 1310 )
 1311 zipped = zip ( * mapped ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/reshape/merge.py in _factorize_keys (lk, rk, sort) 
 1915 rizer = klass ( max ( len ( lk ) , len ( rk ) ) ) 
 1916 
 -> 1917 llab = rizer . factorize ( lk ) 
 1918 rlab = rizer . factorize ( rk ) 
 1919 

 pandas/_libs/hashtable.pyx in pandas._libs.hashtable.Factorizer.factorize () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_labels () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable._unique () 

 pandas/_libs/missing.pyx in pandas._libs.missing.NAType.__bool__ () 

 TypeError : boolean value of NA is ambiguous

In [9]:
#lots of usage date outliers.  need to figure out which instances/companies have the most
